# Librerie


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Apr 13 17:32:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout #Riga interessante
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


In [ ]:
train_path= '/content/drive/MyDrive/Colab Notebooks/Landmark Recognition/data/train'
valid_path= '/content/drive/MyDrive/Colab Notebooks/Landmark Recognition/data/val'
test_path= '/content/drive/MyDrive/Colab Notebooks/Landmark Recognition/data/test'

data_generator = ImageDataGenerator()

train_set = data_generator.flow_from_directory(directory=train_path, target_size=(128,128), class_mode='categorical', batch_size=10)
valid_set = data_generator.flow_from_directory(directory=valid_path, target_size=(128,128), class_mode='categorical', batch_size=10)
test_set = data_generator.flow_from_directory(directory=test_path, target_size=(128,128), class_mode='categorical', batch_size=1, shuffle=False)

Found 11387 images belonging to 100 classes.
Found 1380 images belonging to 100 classes.
Found 1515 images belonging to 100 classes.


# Modello

In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
#STEP 1
x = mobile.layers[-6].output

In [ ]:
#STEP 2
output = Dense(units=100, activation='softmax')(x)

In [ ]:
model = Model(inputs=mobile.input, outputs=output)

In [ ]:
#Vediamo la struttura della rete
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [ ]:
#Sono 88 layers incluso l'input
len(model.layers)

88

In [ ]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [ ]:
#Compile and fit the datasets
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
step_size_train=train_set.n//train_set.batch_size
step_size_train

1138

In [ ]:
step_size_valid=valid_set.n//valid_set.batch_size
step_size_valid

138

In [ ]:
model.fit(x=train_set, steps_per_epoch=step_size_train, validation_data= valid_set, validation_steps=step_size_valid, epochs=30)

Epoch 1/30
1138/1138 [==============================] - 7959s 7s/step - loss: 3.0878 - accuracy: 0.3400 - val_loss: 1.9655 - val_accuracy: 0.5406
Epoch 2/30
1138/1138 [==============================] - 296s 260ms/step - loss: 1.1903 - accuracy: 0.6810 - val_loss: 1.6234 - val_accuracy: 0.6246
Epoch 3/30
1138/1138 [==============================] - 285s 250ms/step - loss: 0.7384 - accuracy: 0.7921 - val_loss: 1.9710 - val_accuracy: 0.5862
Epoch 4/30
1138/1138 [==============================] - 286s 251ms/step - loss: 0.5142 - accuracy: 0.8509 - val_loss: 1.9372 - val_accuracy: 0.5986
Epoch 5/30
1138/1138 [==============================] - 287s 252ms/step - loss: 0.3744 - accuracy: 0.8884 - val_loss: 2.0019 - val_accuracy: 0.6319
Epoch 6/30
1138/1138 [==============================] - 286s 251ms/step - loss: 0.2733 - accuracy: 0.9195 - val_loss: 1.8986 - val_accuracy: 0.6696
Epoch 7/30
1138/1138 [==============================] - 287s 252ms/step - loss: 0.2330 - accuracy: 0.9291 - val_lo

In [ ]:
test_datagen= ImageDataGenerator()

test_set= test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Landmark Recognition/data/test', 
                                             target_size=(128,128), batch_size=1, class_mode='categorical', 
                                             shuffle=False)

Found 1515 images belonging to 100 classes.


In [ ]:
test_set.reset()
test_steps_per_epoch = np.math.ceil(test_set.samples / test_set.batch_size)
print(test_steps_per_epoch)

1515


In [ ]:
predictions = model.predict(x=test_set, steps=test_steps_per_epoch)

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)


[ 0  0  0 ... 99 99 99]


In [ ]:
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys())   

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    

              precision    recall  f1-score   support

       10018       0.30      0.73      0.42        11
       10094       0.61      0.69      0.65        16
       10602       0.72      0.87      0.79        15
       11300       1.00      1.00      1.00        16
       11378       0.56      0.88      0.68        16
       11491       0.78      0.50      0.61        14
       11499       0.86      0.75      0.80        16
       11513       0.67      0.62      0.64        13
        1168       0.54      0.44      0.48        16
       11716       0.67      0.67      0.67        12
       11719       0.93      0.76      0.84        17
       11750       0.93      0.76      0.84        17
       11849       0.67      0.67      0.67        15
       11967       0.58      0.93      0.72        15
       12181       0.77      0.71      0.74        14
       12204       0.94      0.88      0.91        17
       12360       0.60      0.75      0.67        16
       12481       0.67    